In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel(
    "/home/claudio/Downloads/Encuesta_limpieza_22_02_23 vEFRL sinmenores.xlsx",
    index_col=0,
)

In [ ]:
df.shape

In [ ]:
df["Provincia donde vive"].value_counts()

In [ ]:
df["Provincia donde vive"].value_counts(normalize=True)

In [ ]:
miembros_familia = pd.read_excel(
    "inputs/Encuesta_para_inmigrantes_venezolanos_en_Ica_-_all_versions_-_labels_-_2023-02-18-20-40-30.xlsx",
    sheet_name=1,
)

In [ ]:
miembros_familia["Indique el sexo del miembro de familia"].isna().sum()

In [ ]:
miembros_familia["Indique la edad del miembro de familia"].isna().sum()

In [ ]:
(miembros_familia["Indique la edad del miembro de familia"] == 0).sum()

In [ ]:
miembros_familia["Indique la edad del miembro de familia"].describe()

In [ ]:
119 + 6

In [ ]:
4893 - 5014

In [ ]:
miembros_familia = miembros_familia.rename(
    columns={"Indique la edad del miembro de familia": "edad"}
)
miembros_familia["grupo_edad_INEI"] = pd.cut(
    x=miembros_familia["edad"], bins=[-1, 14, 59, 100], labels=["0-14", "15-59", "60+"]
)

In [ ]:
miembros_familia_edad = (
    miembros_familia.groupby(by=["_parent_index", "grupo_edad_INEI"], as_index=False)
    .size()
    .pivot(index="_parent_index", columns="grupo_edad_INEI", values="size")
    .fillna(0)
)
miembros_familia_edad.columns = [
    f"n_miembros_{values}" for values in miembros_familia_edad.columns
]

In [ ]:
df = df.merge(
    right=miembros_familia_edad, how="left", left_index=True, right_on="_parent_index"
).drop(labels="_parent_index", axis=1)
df.loc[:, miembros_familia_edad.columns[1:]] = df[
    miembros_familia_edad.columns[1:]
].fillna(0.0)

In [ ]:
miembros_familia["sexo_edad"] = (
    miembros_familia["Indique el sexo del miembro de familia"].astype(str)
    + "-"
    + miembros_familia["grupo_edad_INEI"].astype(str)
)

In [ ]:
miembros_familia_sexo_edad = (
    miembros_familia.groupby(by=["_parent_index", "sexo_edad"], as_index=False)
    .size()
    .pivot(index="_parent_index", columns="sexo_edad", values="size")
    .fillna(0)
)

In [ ]:
df = df.merge(
    right=miembros_familia_sexo_edad,
    how="left",
    left_index=True,
    right_on="_parent_index",
).drop(labels="_parent_index", axis=1)
df.loc[:, miembros_familia_sexo_edad.columns[1:]] = df[
    miembros_familia_sexo_edad.columns[1:]
].fillna(0.0)

In [ ]:
tabla_edad = (
    df.groupby(["Provincia donde vive"]).agg(
        {"n_miembros_0-14": "sum", "n_miembros_15-59": "sum", "n_miembros_60+": "sum"}
    )
).T

In [ ]:
tabla_edad

In [ ]:
tabla_edad.to_clipboard()

In [ ]:
tabla_sexo = (
    df.groupby(["Provincia donde vive"]).agg(
        {"n_miembros_femenino": "sum", "n_miembros_masculino": "sum"}
    )
).T

In [ ]:
tabla_sexo

In [ ]:
tabla_sexo.to_clipboard()

In [ ]:
aggdict = {col: "sum" for col in miembros_familia_sexo_edad.columns if not "nan" in col}

In [ ]:
tabla_sexo_edad = (df.groupby(["Provincia donde vive"]).agg(aggdict)).T

In [ ]:
tabla_sexo_edad

In [ ]:
tabla_sexo_edad.to_clipboard()

In [ ]:
df["¿Cuál es SU edad?"].min()

In [ ]:
df["¿Cuál es SU edad?"].isna().sum()

In [ ]:
df["n_miembros_menores"].max()

In [ ]:
df["n_miembros_menores"]

In [ ]:
df["n_miembros_menores_bin"] = pd.cut(
    df["n_miembros_menores"], bins=[0.5, 1, 4, 10], labels=["1", "2-3", "4+"]
)

In [ ]:
un_adulto_cargo = df[
    (df["n_miembros_menores"] > 0)
    & (df["n_miembros_adultos"] == 0)
    & (df["n_miembros_mayores"] == 0)
]

In [ ]:
un_adulto_cargo.shape

In [ ]:
tabla_adulto_cargo = (
    un_adulto_cargo.groupby(["Provincia donde vive", "n_miembros_menores_bin"])
    .size()
    .unstack(0)
)

In [ ]:
tabla_adulto_cargo

In [ ]:
tabla_adulto_cargo.to_clipboard()

In [ ]:
df["¿Cuál es SU actual ESTATUS MIGRATORIO?"].unique()

In [ ]:
df["Situación migratoria"] = df["¿Cuál es SU actual ESTATUS MIGRATORIO?"]

In [ ]:
sit_mig = (
    df.groupby(["Provincia donde vive", "Situación migratoria"])
    .size()
    .unstack(0)
    .fillna(0)
)
sit_mig

In [ ]:
sit_mig.to_clipboard()

In [ ]:
df["Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?"].unique()

In [ ]:
contrato_formal = (
    df.groupby(
        [
            "Provincia donde vive",
            "Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?",
        ]
    )
    .size()
    .unstack(0)
    .fillna(0)
)
contrato_formal

In [ ]:
contrato_formal.to_clipboard()

In [ ]:
rel_alim = (
    df.groupby(
        [
            "Provincia donde vive",
            "Actualmente ¿SU TRABAJO SE ENCUENTRA VINCULADO A TEMAS ALIMENTARIOS?",
        ]
    )
    .size()
    .unstack(0)
    .fillna(0)
)
rel_alim

In [ ]:
rel_alim.to_clipboard()

In [ ]:
rel_alim_detalle = (
    df.groupby(["Provincia donde vive", "¿Podría decirnos a que se dedica?"])
    .size()
    .unstack(0)
    .fillna(0)
)
rel_alim_detalle

In [ ]:
rel_alim_detalle.to_clipboard()

In [ ]:
no_rel_alim_detalle = (
    df.groupby(["Provincia donde vive", "¿Podría decirnos a que se dedica?.1"])
    .size()
    .unstack(0)
    .fillna(0)
)
no_rel_alim_detalle

In [ ]:
no_rel_alim_detalle.to_clipboard()

In [ ]:
emprendimiento = (
    df.groupby(
        [
            "Provincia donde vive",
            "Si tiene algun tipo de emprendimiento. ¿Podría decirnos cuál es el rubro de su emprendimiento?",
        ]
    )
    .size()
    .unstack(0)
    .fillna(0)
)
emprendimiento

In [ ]:
emprendimiento.to_clipboard()

In [ ]:
df["Preocupación por no tener alimentos (Leve)"] = df[
    "En los últimos 12 meses ¿Usted u otra persona en su hogar se haya preocupado por no tener suficientes alimentos para comer por falta de dinero u otros recursos?"
]
df["No pudo comer alimentos saludables y nutritivos (v)"] = df[
    "Pensando aún en los últimos 12 meses, ¿hubo alguna vez en que usted u otra persona en su hogar no haya podido comer alimentos saludables y nutritivos  por falta de dinero u otros recursos?"
]
df["Comió poca variedad de alimentos (Moderada)"] = df[
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya comido poca variedad de  alimentos por falta de dinero u otros recursos?"
]
df["Se saltó alguna comida por falta (Moderada)"] = df[
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya tenido que dejar de desayunar, almorzar o cenar porque no había suficiente dinero u otros recursos para obtener alimentos?"
]
df["Comió menos de lo que pensaba que debía (Moderada)"] = df[
    "En los últimos 12 meses Pensando aún en los últimos 12 meses, ¿hubo alguna vez en que usted u otra persona en su hogar haya comido menos de lo que pensaba que debía comer por falta de dinero u otros recursos?"
]
df["Se quedó sin alimentos (Grave)"] = df[
    "En los últimos 12 meses ¿Hubo alguna vez en que su hogar se haya quedado sin alimentos por falta de dinero u otros recursos?"
]
df["Tuvo hambre pero no comió por falta de alientos (Moderada)"] = df[
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya sentido  hambre pero no comió porque no había suficiente dinero u otros recursos para obtener alimentos?"
]
df["Dejo de comer todo un día (Grave)"] = df[
    "En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya dejado de comer todo un día por falta de dinero u otros recursos?"
]

In [ ]:
sit_alim = [
    "Preocupación por no tener alimentos (Leve)",
    "No pudo comer alimentos saludables y nutritivos (v)",
    "Comió poca variedad de alimentos (Moderada)",
    "Comió menos de lo que pensaba que debía (Moderada)",
    "Se saltó alguna comida por falta (Moderada)",
    "Tuvo hambre pero no comió por falta de alientos (Moderada)",
    "Se quedó sin alimentos (Grave)",
    "Dejo de comer todo un día (Grave)",
]

In [ ]:
table = []
for col in sit_alim:
    dff = df[df[col] == "Si"]
    table_col = dff.groupby(["Provincia donde vive"]).size().fillna(0)
    table_col.name = col
    table.append(table_col)
table_sit_alim = pd.concat(table, axis=1).T.fillna(0)
table_sit_alim

In [ ]:
table_sit_alim.to_clipboard()

In [ ]:
df[
    "¿CUÁNDO USTED O ALGUN MIEMBRO DE SU HOGAR SE ENFERMA, A DONDE ACUDE PARA LA ATENCIÓN?"
].unique()

In [ ]:
atnncion_medica = (
    df.groupby(
        [
            "Provincia donde vive",
            "¿CUÁNDO USTED O ALGUN MIEMBRO DE SU HOGAR SE ENFERMA, A DONDE ACUDE PARA LA ATENCIÓN?",
        ]
    )
    .size()
    .unstack(0)
    .fillna(0)
)
atnncion_medica

In [ ]:
df["¿Cual es su nivel educativo?"].unique()

In [ ]:
df["¿Completó el nivel de educacion seleccionado?"].unique()

In [ ]:
edu = (
    df.groupby(
        [
            "Provincia donde vive",
            "¿Cual es su nivel educativo?",
            "¿Completó el nivel de educacion seleccionado?",
        ]
    )
    .size()
    .unstack(0)
    .fillna(0)
)
edu

In [ ]:
edu.to_clipboard()

In [ ]:
df["Tipo de vivienda"].unique()

In [ ]:
df["Tipo de vivienda"].isna().sum()

In [ ]:
tipo_viv = (
    df.groupby(["Provincia donde vive", "Tipo de vivienda"]).size().unstack(0).fillna(0)
)
tipo_viv

In [ ]:
tipo_viv.to_clipboard()

In [ ]:
df.columns[:30]